# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [1]:
import pandas as pd

## imports for Python, Pandas

In [2]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [4]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

name  population
0        Dade       12345
1     Broward       40000
2  Palm Beach       60000
3      Summit        1234
4    Cuyahoga        1337

In [5]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

name  population    state shortname info.governor
0        Dade       12345  Florida        FL    Rick Scott
1     Broward       40000  Florida        FL    Rick Scott
2  Palm Beach       60000  Florida        FL    Rick Scott
3      Summit        1234     Ohio        OH   John Kasich
4    Cuyahoga        1337     Ohio        OH   John Kasich

****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [9]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

[{u'_id': {u'$oid': u'52b213b38594d8a2be17c780'},
  u'approvalfy': 1999,
  u'board_approval_month': u'November',
  u'boardapprovaldate': u'2013-11-12T00:00:00Z',
  u'borrower': u'FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA',
  u'closingdate': u'2018-07-07T00:00:00Z',
  u'country_namecode': u'Federal Democratic Republic of Ethiopia!$!ET',
  u'countrycode': u'ET',
  u'countryname': u'Federal Democratic Republic of Ethiopia',
  u'countryshortname': u'Ethiopia',
  u'docty': u'Project Information Document,Indigenous Peoples Plan,Project Information Document',
  u'envassesmentcategorycode': u'C',
  u'grantamt': 0,
  u'ibrdcommamt': 0,
  u'id': u'P129828',
  u'idacommamt': 130000000,
  u'impagency': u'MINISTRY OF EDUCATION',
  u'lendinginstr': u'Investment Project Financing',
  u'lendinginstrtype': u'IN',
  u'lendprojectcost': 550000000,
  u'majorsector_percent': [{u'Name': u'Education', u'Percent': 46},
   {u'Name': u'Education', u'Percent': 26},
   {u'Name': u'Public Administration, Law, and Ju

In [10]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

_id  approvalfy board_approval_month  \
0  {u'$oid': u'52b213b38594d8a2be17c780'}        1999             November   
1  {u'$oid': u'52b213b38594d8a2be17c781'}        2015             November   

      boardapprovaldate                                 borrower  \
0  2013-11-12T00:00:00Z  FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA   
1  2013-11-04T00:00:00Z                    GOVERNMENT OF TUNISIA   

            closingdate                              country_namecode  \
0  2018-07-07T00:00:00Z  Federal Democratic Republic of Ethiopia!$!ET   
1                   NaN                      Republic of Tunisia!$!TN   

  countrycode                              countryname countryshortname  \
0          ET  Federal Democratic Republic of Ethiopia         Ethiopia   
1          TN                      Republic of Tunisia          Tunisia   

                         ...                           sectorcode source  \
0                        ...                          ET,BS,ES,EP   IBRD   
1                        ...                                BZ,BS   IBRD   

   status  supplementprojectflg  \
0  Active                     N   
1  Active                     N   

                                              theme1  \
0   {u'Percent': 100, u'Name': u'Education for all'}   
1  {u'Percent': 30, u'Name': u'Other economic man...   

                                      theme_namecode themecode   totalamt  \
0  [{u'code': u'65', u'name': u'Education for all'}]        65  130000000   
1  [{u'code': u'24', u'name': u'Other economic ma...     54,24          0   

  totalcommamt                                                url  
0    130000000  http://www.worldbank.org/projects/P129828/ethi...  
1      4700000  http://www.worldbank.org/projects/P144674?lang=en  

[2 rows x 50 columns]

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [17]:
# 1 - countries with the most projects

wb_df = pd.read_json('world_bank_projects.json')
country_counts = wb_df.countryname.value_counts()
country_counts[:11] ## Africa isn't a country

Republic of Indonesia              19
People's Republic of China         19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
Federative Republic of Brazil       9
Name: countryname, dtype: int64

In [51]:
# 2 - top project themes

themes = wb_df.mjtheme_namecode
theme_df = json_normalize(themes[0])
for project in themes:
    x = json_normalize(project)
    theme_df = theme_df.append(x, ignore_index=True)
theme_df.tail()

code                                      name
1496   10                         Rural development
1497    9                         Urban development
1498    8                         Human development
1499    5                     Trade and integration
1500    4  Financial and private sector development

In [50]:
theme_df.describe()
print(theme_df.name.unique())

['Human development' '' 'Economic management'
 'Social protection and risk management' 'Trade and integration'
 'Public sector governance' 'Environment and natural resources management'
 'Social dev/gender/inclusion' 'Financial and private sector development'
 'Rural development' 'Urban development' 'Rule of law']


In [85]:
# 3 - Fill in missing values
# Is there an elegant way to do this?
code_dict = {
            '1' : 'Economic management',
            '2' : 'Public sector governance',
            '3' : 'Rule of law',
            '4' : 'Financial and private sector development',
            '5' : 'Trade and integration',
            '6' : 'Social protection and risk management',
            '7' : 'Social dev/gender/inclusion',
            '8' : 'Human development',
            '9' : 'Urban development',
            '10' : 'Rural development',
            '11' : 'Environment and natural resources management'
            }
             
for k in code_dict.keys():
    theme_df.loc[theme_df.code == k, 'name'] = code_dict[k]

print(theme_df.name.unique())

['Human development' 'Environment and natural resources management'
 'Economic management' 'Social protection and risk management'
 'Trade and integration' 'Public sector governance'
 'Social dev/gender/inclusion' 'Financial and private sector development'
 'Rural development' 'Urban development' 'Rule of law']
